In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [3]:
# создадим функцию для работы:
def init_conn():
    # устанавливаем параметры
    db_config = {'user': 'praktikum_student', # имя пользователя
    'pwd': 'Sdf4$2;d-d30pp', # пароль
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432, # порт подключения
    'db': 'data-analyst-final-project-db'} # название базы данных
    connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

    engine = create_engine(connection_string, connect_args={'sslmode':'require'})
    return engine.connect()

In [4]:
def runSQL(query):
    return pd.io.sql.read_sql(sql=text(query), con = init_conn())

Посмотрим на таблицы:

In [5]:
runSQL('''SELECT * FROM books LIMIT 5''')

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [6]:
runSQL('''SELECT * FROM authors LIMIT 5''')

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [7]:
runSQL('''SELECT * FROM publishers LIMIT 5''')

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [8]:
runSQL('''SELECT * FROM ratings LIMIT 5''')

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [9]:
runSQL('''SELECT * FROM reviews LIMIT 5''')

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Посчитаем, сколько книг вышло после 1 января 2000 года

In [10]:
runSQL('''SELECT COUNT(book_id) FROM books
WHERE publication_date >= '2000-01-01' ''')

,count
0,821


#### После 2000 года вышла 821 книга

### Для каждой книги посчитаем количество обзоров и среднюю оценку

In [17]:
runSQL('''SELECT b.book_id, b.title, COUNT(DISTINCT(re.review_id)) AS review, ROUND(AVG(ra.rating)) AS avg_rating FROM books AS b
LEFT join reviews as re on b.book_id = re.book_id
join ratings as ra on b.book_id = ra.book_id
GROUP BY b.book_id
ORDER BY avg_rating, review
''')

,book_id,title,review,avg_rating
0,303,Harvesting the Heart,2,2.0
1,316,His Excellency: George Washington,2,2.0
2,371,Junky,2,2.0
3,915,The World Is Flat: A Brief History of the Twen...,3,2.0
4,794,The Mermaid Chair,3,2.0
...,...,...,...,...
995,423,Maus I: A Survivor's Tale: My Father Bleeds Hi...,4,5.0
996,203,Drums of Autumn (Outlander #4),4,5.0
997,415,Magic Bites (Kate Daniels #1),4,5.0
998,741,The Green Mile,4,5.0


### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [12]:
runSQL('''SELECT p.publisher, COUNT(b.book_id) AS sum_book FROM books AS b
INNER JOIN publishers AS p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
group by p.publisher
order by sum_book DESC
LIMIT 1
''')

,publisher,sum_book
0,Penguin Books,42


#### Издательство Penguin Books выпустило наибольшее число книг толще 50 страниц

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [13]:
runSQL('''SELECT i.author, ROUND(AVG(i.sum_rating),2) as avg_rating
FROM
  (SELECT a.author, AVG(r.rating) AS sum_rating, b.book_id FROM books AS b
INNER JOIN ratings AS r ON b.book_id = r.book_id
INNER JOIN author AS a ON b.author_id = a.author_id
group by a.author,b.book_id
order by sum_rating DESC
) AS i
RIGHT OUTER JOIN
  (SELECT b.book_id , COUNT(ra.rating_id) as rating_count FROM books AS b
inner join ratings AS ra ON b.book_id = ra.book_id
GROUP BY b.book_id
HAVING COUNT(ra.rating_id) > 50
ORDER BY COUNT(ra.rating_id)) AS c ON i.book_id = c.book_id
GROUP BY author
LIMIT 1

''')

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.28


#### Самая высокая средняя оценка у книг Роулинг с иллюстрациями Мари Грандпре

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [14]:
runSQL('''SELECT AVG(R)
FROM
(SELECT username, COUNT(review_id) AS R
FROM reviews
WHERE username IN (SELECT username
FROM ratings
GROUP BY username
HAVING COUNT(rating_id) > 48)
GROUP BY username) as a; 
   ''')


,avg
0,24.0


#### В среднем пользователи с 48 оценками и больше проводят 24 обзора

## Выводы:  
    После 2000 года вышла 821 книга.   
    Количество обзоров для книг варьируется от 4 до 55, рейтинг от 2 до 5.   
    Издательство Penguin Books выпустило наибольшее число книг толще 50 страниц   
    Самая высокая средняя оценка у книг Роулинг отчатси благодаря иллюстрациям Мари Грандпре   
    В среднем пользователи с 48 оценками проводят 24 обзора  